# CLARA A3 joint histograms

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from c3s_eqc_automatic_quality_control import download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

## Set parameters

In [ ]:
# Time
year_start = 1979
year_stop = 2020

# Region
lon_slice = slice(-180, 180)
lat_slice = slice(-60, 60)

# Bins
cot_slices = [slice(None, 1.3), slice(1.3, 9.4), slice(9.4, None)]
ctp_slices = [slice(None, 310), slice(310, 680), slice(680, None)]

## Define request

In [ ]:
collection_id = "satellite-cloud-properties"

request = {
    "product_family": "clara_a3",
    "origin": "eumetsat",
    "variable": "joint_cloud_property_histogram",
    "climate_data_record_type": "thematic_climate_data_record",
    "time_aggregation": "monthly_mean",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
}

## Define functions to cache

In [ ]:
def infer_label(sel_slice, var_name, units=""):
    if units:
        units = f" {units.strip()}"
    if sel_slice.start is None:
        return f"{var_name} < {sel_slice.stop}{units}"
    if sel_slice.stop is None:
        return f"{var_name} ≥ {sel_slice.start}{units}"
    return f"{sel_slice.start}{units} ≤ {var_name} < {sel_slice.stop}{units}"


def reindex(ds, cot_slices, ctp_slices, lon_slice, lat_slice):
    ds = utils.regionalise(ds, lat_slice=lat_slice, lon_slice=lon_slice)
    datasets = []
    for cot_index, cot_slice in enumerate(cot_slices):
        for ctp_index, ctp_slice in enumerate(ctp_slices):
            ds_limit = ds.sel(
                hist2d_cot_bin_centre=cot_slice,
                hist2d_ctp_bin_centre=ctp_slice,
            )
            ds_limit = ds_limit.sum(
                ["hist_phase", "hist2d_ctp_bin_centre", "hist2d_cot_bin_centre"],
                keep_attrs=True,
            )
            ds_limit = ds_limit.expand_dims(
                COT=[infer_label(cot_slice, "COT")],
                CTP=[infer_label(ctp_slice, "CTP", "hPa")],
            )
            ds_limit = ds_limit.assign_coords(
                cot_index=("COT", [cot_index]), ctp_index=("CTP", [ctp_index])
            )
            datasets.append(ds_limit)
    ds = xr.combine_by_coords(datasets)
    ds = ds.sortby(["cot_index", "ctp_index"]).drop_vars(["cot_index", "ctp_index"])
    ds = ds.drop_dims(["hist2d_cot_bin_border", "hist2d_ctp_bin_border"])
    return ds

## Download and transform

In [ ]:
ds = download.download_and_transform(
    collection_id,
    request,
    chunks={"year": 1},
    transform_func=reindex,
    transform_func_kwargs={
        "lon_slice": lon_slice,
        "lat_slice": lat_slice,
        "cot_slices": cot_slices,
        "ctp_slices": ctp_slices,
    },
    drop_variables=["lat_bnds", "lon_bnds", "time_bnds"],
)

## Quick and dirty plot: Normalized bins

In [ ]:
da = ds["hist2d_cot_ctp"].sum("time") / ds["hist2d_cot_ctp"].sum(["COT", "CTP", "time"])
facet = plot.projected_map(
    da,
    col="COT",
    row="CTP",
    cmap="ocean_r",
    levels=np.linspace(0, 1, 13),
    projection=ccrs.PlateCarree(central_longitude=180),
    figsize=(14, 6),
    cbar_kwargs={"label": ""},
)
_ = facet.fig.suptitle(f"CLARA A3\n{year_start}-01 - {year_stop}-12", y=1.1)